In [ ]:
"""
Dataset: https://data.mendeley.com/datasets/4drtyfjtfy/1

Use the given dataset and the keras package to build a neural network image classifier that categorizes an image as rainy, sunny, cloudy or sunrise.
"""

import os

from keras.layers import Activation, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import get_file

# New Section

In [ ]:
imagegen = ImageDataGenerator(rescale=1. / 255., rotation_range=30, horizontal_flip=True, validation_split=0.1)

path_to_zip = get_file('weatherData.zip', origin="https://static.junilearning.com/ai_level_2/weatherData.zip", extract=True)
#PATH = os.path.join(os.path.dirname(path_to_zip), 'weatherData')
PATH = os.path.join(os.path.dirname(path_to_zip), 'weatherData_extracted/weatherData')
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')
test_dir = os.path.join(PATH, 'test')

52863186/52863186 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
train_generator = imagegen.flow_from_directory(train_dir, class_mode="categorical", shuffle=True, batch_size=128, target_size=(224, 224))
validation_generator = imagegen.flow_from_directory(validation_dir, class_mode="categorical", shuffle=True, batch_size=128, target_size=(
	224, 224))
test_generator = imagegen.flow_from_directory(test_dir, class_mode="categorical", shuffle=False, batch_size=128, target_size=(224, 224))

Found 320 images belonging to 4 classes.
Found 40 images belonging to 4 classes.
Found 40 images belonging to 4 classes.


In [ ]:
# build a sequential model
# Building the model
model = Sequential()

# 3 convolutional layers
model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 2 hidden layers
model.add(Flatten())
model.add(Dense(128))
model.add(Activation("relu"))

model.add(Dense(128))
model.add(Activation("relu"))

# The output layer with 4 neurons (1 per class)
model.add(Dense(4))
model.add(Activation("softmax"))

# compile model
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
# fit on data for 10 epochs
model.fit(train_generator, epochs=10, validation_data=validation_generator)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 56s 17s/step - accuracy: 0.3173 - loss: 1.4513 - val_accuracy: 0.3750 - val_loss: 1.1611
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 38s 14s/step - accuracy: 0.4047 - loss: 1.1818 - val_accuracy: 0.6250 - val_loss: 0.8690
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 40s 12s/step - accuracy: 0.5927 - loss: 0.9319 - val_accuracy: 0.7250 - val_loss: 0.6881
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 39s 16s/step - accuracy: 0.6458 - loss: 0.7566 - val_accuracy: 0.7250 - val_loss: 0.5303
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 40s 12s/step - accuracy: 0.5770 - loss: 0.8642 - val_accuracy: 0.7750 - val_loss: 0.4472
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 37s 11s/step - accuracy: 0.7035 - loss: 0.6848 - val_accuracy: 0.8250 - val_loss: 0.3713
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 41s 11s/step - accuracy: 0.7138 - loss: 0.5984 - val_accuracy: 0.7750 - val_loss: 0.5010
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 37s 14s/step - accuracy: 0.7203 - loss: 0.6629 - val_accuracy: 0.9250 - val_loss: 0.3905


In [ ]:
accuracy = model.evaluate(test_generator)[1]
print(accuracy)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.7750 - loss: 0.5229
0.7749999761581421


In [ ]:
examples_datagen = ImageDataGenerator(rescale=1. / 255)
examples_generator = examples_datagen.flow_from_directory(
	PATH,
	target_size=(224, 224),
	classes=['examples'],
	shuffle=False)
examples_generator.reset()
preds = model.predict(examples_generator)
predicted_classes = []
for pred in preds:
	predicted_class = -1
	max = -1
	for i in range(len(pred)):
		if pred[i] > max:
			max = pred[i]
			predicted_class = i
	predicted_classes.append(predicted_class)

for pred, name in zip(predicted_classes, examples_generator.filenames):
	print(f'Filename: {name}')
	print(f'Label: {list(train_generator.class_indices.items())[pred][0]}')
	print()

Found 4 images belonging to 1 classes.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step
Filename: examples/cloudy.jpg
Label: rainy

Filename: examples/rainy.jpg
Label: rainy

Filename: examples/sunrise.jpg
Label: sunrise

Filename: examples/sunshine.jpg
Label: sunny

